In [2]:
!pip install cirq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 79.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 60.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 76.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 80.3 MB

In [3]:
# import cirq
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [4]:
datagen=ImageDataGenerator(
rescale=1.0/255,
rotation_range=40,
width_shift_range=0.2, 
height_shift_range=0.2, 
zoom_range=0.2, 
shear_range=0.2, 
fill_mode="nearest",
validation_split=0.2,
horizontal_flip=True
)
df="/kaggle/input/lung-cancer-histopathological-images"
train_data=datagen.flow_from_directory(
df,
target_size=(256,256),
batch_size=32,
# class_mode=None
# validation_split=0.2,
subset="training"
)
val_data=datagen.flow_from_directory(
df,
target_size=(256,256),
batch_size=32,
# class_mode=None
# validation_split=0.2,
subset="validation"
)


Found 12000 images belonging to 3 classes.
Found 3000 images belonging to 3 classes.


# # CNN

In [5]:
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Layer,BatchNormalization,Dropout,Input,ReLU
from keras.models import Model
from keras.regularizers import l2
from keras.initializers import HeNormal
from keras.callbacks import EarlyStopping

In [6]:
early_stopping=EarlyStopping(
monitor="val_loss",
patience=5,
verbose=1)

In [7]:
inputs=Input(shape=(256,256,3))
C=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
C=BatchNormalization()(C)
C=Dropout(0.5)(C)
C=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)
C=Conv2D(64,(3,3),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
C=ReLU()(C)
C=BatchNormalization()(C)
C=Dropout(0.5)(C)
C=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)
C=Conv2D(128,(3,3),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
C=ReLU()(C)
C=BatchNormalization()(C)
C=Dropout(0.5)(C)
C=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)
C=Conv2D(256,(3,3),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
C=ReLU()(C)
C=BatchNormalization()(C)
C=Dropout(0.5)(C)
C=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)
C=Conv2D(556,(3,3),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
C=ReLU()(C)
C=BatchNormalization()(C)
C=Dropout(0.5)(C)
C=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)
C=Flatten()(C)
D=Dense(64,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
D=Dropout(0.5)(D)
D=ReLU()(D)
D=Dense(32,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dropout(0.5)(D)
D=ReLU()(D)            
outputs=Dense(3,activation="softmax")(D)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss="categorical_crossentropy",metrics=['accuracy'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32, 32, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 16, 16, 556)    │     1,281,580 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_3 (ReLU)                  │ (None, 16, 16, 556)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 16, 16, 556)    │         2,22

 Total params: 3,953,759 (15.08 MB)

 Trainable params: 3,951,687 (15.07 MB)

 Non-trainable params: 2,072 (8.09 KB)

In [8]:
model.fit(train_data,epochs=50,validation_data=val_data,callbacks=[early_stopping])

Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1729159974.694304     139 service.cc:145] XLA service 0x7a2264017c50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729159974.694358     139 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1729159974.694363     139 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


  1/375 ━━━━━━━━━━━━━━━━━━━━ 3:21:19 32s/step - accuracy: 0.2812 - loss: 27.0201

I0000 00:00:1729159995.987997     139 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


375/375 ━━━━━━━━━━━━━━━━━━━━ 382s 934ms/step - accuracy: 0.4646 - loss: 20.5553 - val_accuracy: 0.6027 - val_loss: 5.5536
Epoch 2/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 272s 714ms/step - accuracy: 0.5064 - loss: 4.4548 - val_accuracy: 0.6450 - val_loss: 2.5149
Epoch 3/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 271s 712ms/step - accuracy: 0.8089 - loss: 1.9179 - val_accuracy: 0.4850 - val_loss: 9.7482
Epoch 4/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 271s 712ms/step - accuracy: 0.8324 - loss: 1.4176 - val_accuracy: 0.6243 - val_loss: 4.2157
Epoch 5/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 271s 713ms/step - accuracy: 0.8232 - loss: 1.2106 - val_accuracy: 0.5847 - val_loss: 3.0965
Epoch 6/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 271s 713ms/step - accuracy: 0.8329 - loss: 1.1438 - val_accuracy: 0.6427 - val_loss: 2.8142
Epoch 7/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 271s 712ms/step - accuracy: 0.8492 - loss: 1.0329 - val_accuracy: 0.3333 - val_loss: 10.6691
Epoch 7: early stopping
